# Word2Vec 를 통한 위픽 개인화 테스트



In [1]:
import requests
import time
import urllib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import json
import elasticsearch
import csv
import pickle
from elasticsearch.helpers import bulk
import re
import glob
import os
from datetime import timezone, timedelta, datetime
from pymongo import MongoClient
import pymongo
import pandas as pd
import numpy as np
from operator import itemgetter

In [2]:
### twiceSpark1
es_url = '10.102.50.47:9200'

In [3]:
es = elasticsearch.Elasticsearch(es_url)

## ES 관련 루틴


In [4]:
def es_search_dids_for_user(user_id, day_limit):
    """
    user_id의 모든 v 가져오기
    day_limit 이전 것만 가져온다.
    return
    - 1st: v의 set
    - 2nd: 확장 정보 (v, rgtime, slot)
    """
    res = es.search(index='wepick_seq', 
                body={
                  "query": {
                    "bool": {
                      "must": { 
                        "term": {"u":user_id}
                      },
                      "filter": {
                        "range" : {
                          "rgtime" : {
                          "lt" : day_limit
                          }
                        }
                      }
                    }
                  },
                  "size": 64,
                  "sort": {"rgtime":"desc"}
                                }
               )
    if res['hits']['total'] > 0:
        until_dt = pd.to_datetime(day_limit).to_pydatetime()
        filtered = []
        for hit in res['hits']['hits']:
            filtered.append((hit['_source']['v'], hit['_source']['rgtime'], hit['_source']['slot']))
        return set(map(lambda x: x[0], filtered)), filtered
    return None, None

In [5]:
def es_gather_word2vec_dids(dids):
    """
    dids로부터, word2vec을 모은다.
    - 모아서, average pooling 실시
    return
    - vector normalized by L2-norm
    """
    res = es.search(index='deal_word2vec', 
                body={
                    'from':0, 'size': len(dids),
                    "_source": ["values"],
                    'query': {
                        'ids': {'values': dids }
                        }                        
                    }
               )
    mat = []
    for hit in res['hits']['hits']:
        vec = np.array(hit['_source']['values'])
        if len(vec) > 0:
            mat.append(vec)
    vec = np.mean(np.vstack(mat), axis=0)
    vec /= np.sqrt(np.sum(vec**2))
    return vec


In [6]:
def es_gather_word2vec_wepick(dids):
    """
    dids로부터, word2vec을 모은다.
    return
    - dids: unit-length w2v (normalized by L2-norm)
    """
    res = es.search(index='deal_word2vec', 
                body={
                    'from':0, 'size': len(dids),
                    "_source": ["values", "v"],
                    'query': {
                        'ids': {'values': dids }
                        }                        
                    }
               )
    dic = {}
    for hit in res['hits']['hits']:
        did = hit['_source']['v']
        vec = np.array(hit['_source']['values'])
        if len(vec) > 0:
            vec /= np.sqrt(np.sum(vec**2))
            dic[did] = vec
    return dic


In [7]:
def es_read_wepick_setting(dt, start_slot=20):
    """
    위픽 세팅 로딩
    """
    res = es.search(index='wepick_setting_ext', 
                body={
                    'query': {
                        'term': {'dt': dt }
                        }                        
                    }
               )
    if res['hits']['total'] > 0:
        dic = {}
        vec = []
        for s in res['hits']['hits'][0]['_source']['settings']:
            if s['slot'] >= start_slot:
                dic[s['slot']] = s['did']
                vec.append(s['did'])
        return vec, dic
    return None, None

In [8]:
def es_scan_extra_by_dids(dids):
    """
    dids로부터, mn, tn1를 가져온다.
    """
    res = es.search(index='dealinfos', 
                body={
                    'from':0, 'size': len(dids),
                    "_source": ["mn", "tn1", "did"],
                    'query': {
                        'ids': {'values': dids }
                        }                        
                    }
               )
    dic = {}
    for hit in res['hits']['hits']:
        dic[hit['_source']['did']] = (hit['_source']['mn'], hit['_source']['tn1'])
    return dic

### mongoDB for ActionInfos2

In [9]:
client = MongoClient(host='35.190.239.204', port=27017, username='praha_read', password='praha!@#', authSource='praha')

db = client['praha']

col = db['memberActionInfos2']

In [10]:
def mg_get_ordered_dids(mid, lt_day="20180411", limit=32):
    """
    구매한 딜들을 조회
    """
    result = col.find({"mid":mid, 'ft.o':{"$ne":[]}, 'day':{"$lt":lt_day}}, {'day':1, 'ft.o': 1, '_id':0}).sort('day', pymongo.DESCENDING).limit(limit)
    out = set()
    for res in result:
        out.update(list(map(lambda x: x['did'], res['ft']['o'])))
    return out

In [11]:
def mg_get_clicked_dids(mid, lt_day="20180411", limit=32, use_search_induced_click=False):
    """
    클릭한 딜들을 조회
    """
    result = col.find({"mid":mid, 'ft.c':{"$ne":[]}, 'day':{"$lt":lt_day}}, {'day':1, 'ft.c': 1, '_id':0}).sort('day', pymongo.DESCENDING).limit(limit)
    out = set()
    for res in result:
        out.update(list(map(lambda x: x['did'], 
                            res['ft']['c'] if use_search_induced_click == False else filter(lambda x: x['s'] != '', res['ft']['c'])
                           )))
    return out

### Wepick Setting load

In [12]:
# 2018-04-11 21 시의 위픽 세팅 로딩
wepick_setting, wepick_dic = es_read_wepick_setting('2018-04-11 21')

In [13]:
wepick_slot_dic = dict(zip(wepick_dic.values(), wepick_dic.keys()))

## Ranking 관련

In [14]:
def calc_rank(user_profile, deal_profile_dic):
    """
    user_profile, deal_profile의 inner product를 통한 점수 계산 및 정렬 (내림차순)
    """
    scores = []
    for did, deal_profile in deal_profile_dic.items():
        scores.append((did, np.inner(user_profile, deal_profile)))
    return sorted(scores, key=itemgetter(1), reverse=True)

In [15]:
def get_refined_scores(scores, extra_dic):
    refined_scores = []
    for did, score in scores:
        if did in extra_dic:
            refined_scores.append((score, did, extra_dic[did][0], extra_dic[did][1]))
        else:
            refined_scores.append((score, did, "", ""))
    return refined_scores

In [16]:
def print_result(out, wepick_slot_dic):
    for s, did, title, cate in out:
        org_slot = wepick_slot_dic[did] if did in wepick_slot_dic else -1
        print((s, did, title, org_slot, cate))
            

### deal_profile loading

In [17]:
# 위픽 세팅에 따른 딜들에 대한 deal_profile을 생성
deal_profile_dic = es_gather_word2vec_wepick(wepick_setting)

### 3월 11 -  4월 10일까지 위픽 클릭 데이터에 대해 구성한 user_profile에 대한 테스트

In [18]:
deals_user_viewed, ex = es_search_dids_for_user(1000007, '2018-04-11')

In [19]:
user_profile = es_gather_word2vec_dids(list(deals_user_viewed))

In [20]:
scores = calc_rank(user_profile, deal_profile_dic)

In [21]:
extra_dic = es_scan_extra_by_dids(wepick_setting)

In [22]:
out = get_refined_scores(scores, extra_dic)

In [23]:
print_result(out, wepick_slot_dic)

(0.570319645878629, 3515524, '[무료배송] 롱티/티셔츠/원피스', 45, '티셔츠')
(0.5492949691109743, 3512593, '[무료배송] 봄 아동복 브랜드 연합전', 36, '아동공용의류')
(0.5152828672370329, 3525317, '[무료배송] 빅사이즈/원피스/롱티', 28, '원피스')
(0.5056083153404339, 3522402, '[무료배송] 에비수 본사특가 20%쿠폰', 44, '티셔츠')
(0.5023033625224077, 3525500, '[하객패션] 포커스 봄구성완벽해', 74, '티셔츠')
(0.4644707094849176, 3527477, '[투데이특가] 니트/가디건/원피스 외', 61, '원피스')
(0.4422997458733418, 3527575, '[무료배송] 프롬유 ~20%할인쿠폰', 55, '티셔츠')
(0.40502225413567416, 3514459, '[심야특가] 파파야 여성 의류 모음전', 32, '티셔츠')
(0.32791514351610584, 3528363, '[하객패션] 백화점 잡화 267종! +20%', 27, '벨트')
(0.2936802497671649, 3524547, '[투데이특가] 1+ 한우 제비추리150g', 58, '한우')
(0.26202071748506295, 3522395, '[롯데] 르까프 아동/성인 빅세일', 46, '남성 티셔츠/상의 기타')
(0.25462440793404334, 3527569, '[투데이특가] 아디다스 그래픽스케일', 91, '반팔 티셔츠')
(0.2501394346573817, 3512215, '[6천원쿠폰] 기습쿠폰전 오늘마지막!', 30, '색조메이크업')
(0.2439982042589318, 3515690, '[투데이특가] 닥스셔츠 긴/반팔 BEST', 81, '셔츠/남방')
(0.23662753644035836, 3532677, '[게릴라특가] 궁중 너비아니 4팩', 82, '가공육/양념육류')
(0.

### 구매 did로 부터 랭킹 테스트

- 구매 did 들의 word2vec을 사용

In [24]:
dids = mg_get_ordered_dids(1000007, limit=32)

In [25]:
user_profile = es_gather_word2vec_dids(list(dids))

In [26]:
scores = calc_rank(user_profile, deal_profile_dic)

In [27]:
out = get_refined_scores(scores, extra_dic)

In [28]:
print_result(out, wepick_slot_dic)

(0.3363883512659658, 3522395, '[롯데] 르까프 아동/성인 빅세일', 46, '남성 티셔츠/상의 기타')
(0.2962912952096999, 3505573, '[무료배송] 까만색 잇몸치약 130g ', 29, '칫솔/치약')
(0.2854306974545242, 3515690, '[투데이특가] 닥스셔츠 긴/반팔 BEST', 81, '셔츠/남방')
(0.28349112392163545, 3518630, '[투데이특가] 꺾어먹는 비요뜨 12개', 79, '두유/우유')
(0.25622797329069014, 3519047, '[투데이특가] 풀무원 간편국15+5입', 31, '즉석밥/국/카레')
(0.25358373613821905, 3524294, '[투데이특가] 여심저격 클루나드시계', 83, '패션시계')
(0.2525332526576739, 3528363, '[하객패션] 백화점 잡화 267종! +20%', 27, '벨트')
(0.23782544874998923, 3522402, '[무료배송] 에비수 본사특가 20%쿠폰', 44, '티셔츠')
(0.23201802270369787, 3524547, '[투데이특가] 1+ 한우 제비추리150g', 58, '한우')
(0.23156091573061713, 3119252, '[역시트롬] LG 드럼세탁기 모음', 60, '드럼세탁기')
(0.23099094495665168, 3521785, '[쿠폰할인] 중고폰 노트5/S7/엣지', 51, '공기계-미사용/미개봉')
(0.2254796971959036, 3529165, '[하객패션] 락피쉬 18년S/S+20%쿠폰!', 70, '여성단화')
(0.22124419556650265, 3512421, '[투데이특가] 휴대용 핸디 선풍기!', 85, '휴대용 선풍기')
(0.2192870436658886, 3527824, '[투데이특가] 우리밀 아이국수 8인분', 59, '아기간식/아기음료')
(0.21536448836925087, 3464309, '[쿠

### 클릭 did로 부터 랭킹 테스트

- 유저가 클릭한 did 들의 word2vec을 사용
- use_search_induced_click==True 면, 검색후 클릭된 did만 가져온다.

In [29]:
dids= mg_get_clicked_dids(1000007, limit=32, use_search_induced_click=True)

In [30]:
user_profile = es_gather_word2vec_dids(list(dids))

In [31]:
scores = calc_rank(user_profile, deal_profile_dic)

In [32]:
out = get_refined_scores(scores, extra_dic)

In [33]:
print_result(out, wepick_slot_dic)

(0.4797754883108316, 3521785, '[쿠폰할인] 중고폰 노트5/S7/엣지', 51, '공기계-미사용/미개봉')
(0.3884136782550534, 3504137, '[리빙위크] 비즈니스보루네오 소파', 78, '소파')
(0.33398995150777544, 3464309, '[쿠폰할인] LG 공기청정기 AS181DAW', 57, '공기청정기')
(0.3276135397874731, 3068897, '[가전쿠폰] LG 13kg 통돌이세탁기', 37, '일반 세탁기')
(0.3216869922914043, 2258611, '[사은품증정] 리빙웰 에어프라이어', 84, '튀김기')
(0.3023593388373891, 3511172, '[컬러풀] GTX1060 6GB 그래픽카드', 80, '그래픽카드')
(0.2830360070050044, 3524547, '[투데이특가] 1+ 한우 제비추리150g', 58, '한우')
(0.28020076086544904, 3061867, '[추가쿠폰] 삼성 냉장고 RH81K8050SA', 50, '양문형 냉장고')
(0.27519410410665174, 3512421, '[투데이특가] 휴대용 핸디 선풍기!', 85, '휴대용 선풍기')
(0.26368009766893546, 3532677, '[게릴라특가] 궁중 너비아니 4팩', 82, '가공육/양념육류')
(0.2618446704243125, 3527569, '[투데이특가] 아디다스 그래픽스케일', 91, '반팔 티셔츠')
(0.25468799403656706, 3544419, '[게릴라특가] 봄맞이 카페트 150x200', 71, '카페트/러그')
(0.24962228799236605, 3521723, '[투데이특가] 샤오미 공기청정기', 25, '공기청정기')
(0.24719623436934426, 3119252, '[역시트롬] LG 드럼세탁기 모음', 60, '드럼세탁기')
(0.24370336644632376, 3541064, '[위메프] 10만 